<a href="https://colab.research.google.com/github/yangliupku/cs336_assignment5_alignment/blob/main/notebooks/Qwen_model_MATH_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install math-verify[antlr4_13_2]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.9.3
    Uninstalling antlr4-python3-runtime-4.9.3:
      Successfully uninstalled antlr4-python3-runtime-4.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
omegaconf 2.3.0 requires antlr4-python3-runtime==4.9.*, but you have antlr4-python3-runtime 4.13.2 which is incompatible.


In [ ]:
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset
from math_verify import parse, verify
import pandas as pd
from tqdm import tqdm
import torch
import time

## Sequetial processing

In [ ]:
ds = load_dataset("qwedsacf/competition_math")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-7320a6f3aba8eb(…):   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [ ]:
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-Math-1.5B")
# device = 0 if torch.cuda.is_available() else -1
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B")
# # tokenizer.padding_side = "left"  # Critical for decoder-only models
# # tokenizer.pad_token = tokenizer.eos_token  # Set pad token
# pipe = pipeline(
#     "text-generation",
#     model="Qwen/Qwen2.5-Math-1.5B",
#     tokenizer=tokenizer,
#     device=0 if torch.cuda.is_available() else -1,
#     torch_dtype=torch.float16
# )

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
for i in range(10):
  question = ds['train'][i]['problem']
  gt_answer = parse(ds['train'][i]['solution'])
  prompt  = f"""A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
  User: {question}
  Assistant: <think>"""
  model_response = pipe(prompt, max_new_tokens=1024)
  model_answer = parse(model_response[0]['generated_text'])
  is_correct = verify(gt_answer, model_answer)
  print(f"-------problem {i}--------")
  print(f"Question: {question}")
  print(f"model response: {model_response[0]['generated_text']}")
  print(f"GT Answer: {gt_answer}")
  print(f"Model Answer: {model_answer}")
  print(f"Is correct: {is_correct}")



-------problem 0--------
Question: Let \[f(x) = \left\{
\begin{array}{cl} ax+3, &\text{ if }x>2, \\
x-5 &\text{ if } -2 \le x \le 2, \\
2x-b &\text{ if } x <-2.
\end{array}
\right.\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).
model response: A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
  User: Let \[f(x) = \left\{
\begin{array}{cl} ax+3, &\text{ if }x>2, \\
x-5 &\text{ if } -2 \le x \le 2, \\
2x-b &\text{ if } x <-2.
\end{array}
\right.\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn with

## Batch processing

In [ ]:
batches = list(ds['train'].iter(batch_size=8))

In [ ]:
def get_prompt(question):
  return f"""A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
  User: {question}
  Assistant: <think>"""

result = []
for batch in tqdm(batches, "processing batches"):
  batch_prompts = [get_prompt(problem) for problem in batch['problem']]
  batch_responses = pipe(batch_prompts, max_new_tokens=1024)
  batch_result = []
  for i in range(len(batch)):
    question = batch['problem'][i]
    solution = batch['solution'][i]
    gt_answer = parse(batch['solution'][i])
    model_response = batch_responses[i][0]['generated_text']
    model_answer = parse(model_response)
    is_correct = verify(gt_answer, model_answer)
    batch_result.append({
        'question': question,
        'solution': solution,
        'gt_answer': gt_answer,
        'model_response': model_response,
        'model_answer': model_answer,
        'is_correct': is_correct
    })
  result.extend(batch_result)

processing batches:   1%|▏         | 21/1563 [32:55<40:17:50, 94.08s/it] 


KeyboardInterrupt: 

8